In [ ]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime

# Reservoir characteristics.



First we download the relevant data from the spreadsheet.

In [ ]:
# First we download the key data
key_parameters = pd.read_excel('data/Conowingo_data.xlsx', sheet_name='Reservoir characteristics')

# Access the DataFrame
print(key_parameters)

Now we can have the key lake stats when it is full, and **convert them in the same units immediately** (m, m2, m3).

In [ ]:
full_lake_area =  key_parameters.iloc[2, key_parameters.columns.get_loc('Value')] * 100**2
full_lake_volume = key_parameters.iloc[1, key_parameters.columns.get_loc('Value')] * 100**3


# Defining a reservoir


A reservoir has many characteristics that need to be represented:
*   How storage volume, surface area and height of water in the reservoir are related. This is important for hydropower, recreation, but also for withdrawing water from the reservoir: water is withdrawn from a physical pipe situated at a certain depth (depth being calculated from the water level when the reservoir lake is full).
*   We need to know what water uses depend on the reservoir.


For this, we will construct them as an object in Python. First we declare the class, then associated functions. **Our key assumption here is that lake area increases linearly with the height of water in the lake**.

To note:


*   All units converted to basic metric units
*   Water balance will track storage volume so basic function convert that into area and height.



In [ ]:
class Reservoir:

  # Define attributes specific to each Reservoir object
  def __init__(self, name, full_lake_area, full_lake_volume, dead_storage=0):
    self.name = name
    self.full_lake_area = full_lake_area # Surface area of the lake at full capacity in square metres (m2)
    self.full_lake_volume = full_lake_volume # Volume of water at full capacity in cubic metres (m3)

    # Attributes deduced from others
    # Assuming a simplified linear relationship between water level and surface area
    self.total_lake_depth = self.full_lake_volume / (self.full_lake_area / 2)
    self.initial_storage = 0.9 * self.full_lake_volume  # Initialize storage at 90% full

    # Optional attribute with default value of 0: dead storage volume, the volume below which release is impossible
    # By default dead storage is empty storage
    self.dead_storage = dead_storage

    # Initialise demands
    self.demand_on_site = []
    self.demand_downstream = []

  # Method to calculate the current surface area based on current volume (in m3)
  def get_surface_area(self,  volume):
    # Make sure volume is bounded
    current_volume = min(self.full_lake_volume, max(0, volume))
    return math.sqrt(2*current_volume*self.full_lake_area/(self.total_lake_depth))

  # Method to calculate the current water height based on the current volume (in m3)
  def get_height(self, volume):
    # Make sure volume is bounded
    return 2* min(self.full_lake_volume, max(0, volume))  / self.get_surface_area(volume)

  # Method to add an on-site demand
  def add_on_site_demand(self, demand):
    self.demand_on_site.append(demand)
    return None

  # Method to add a demand dowsntream of the reservoir
  def add_downstream_demand(self, demand):
    self.demand_downstream.append(demand)
    return None

Let us create a reservoir object with specified values, see slide 2 in the accompanying slides for this tutorial.

In [ ]:
# Create a reservoir object with the specified values
res = Reservoir("Conowingo", full_lake_area, full_lake_volume)

# Print characteristics of reservoir object
print(res.__dict__)

In [ ]:
# Let us modify the reservoir to reflect actual dead storage
res.dead_storage = key_parameters.iloc[0, key_parameters.columns.get_loc('Value')] * 100**3
print('Dead storage is now ' + str(int(res.dead_storage  / 100**3)) + 'hm3.')

# Water balance over a single time step

Let us model inflows and releases (see still slide 2 from tutorial slides). In practice, here we assume that:

1.   The ecological demand is served first (if possible).
2.   Other demands take water directly from the reservoir in the limit of what remains.
3.   Then excess water is spilled and added to the releases.

In [ ]:
def single_step(reservoir, storage_beg, inflows, site_demand, downstream_demand):

    '''
    Note all in m3.
    :param reservoir: Object of the Reservoir class
    :param storage_beg: Initial storage at the beginning of the time step (m3)
    :param inflows: Inflows over the time step (m3)
    :param site_demand: Demand for withdrawal from reservoir over the time step (m3)
    :param downstream_demand: Demand for release for downstream use over the time step (m3)
    :return: storage_end (end of time step storage, m3)
    :return: release (amount of water released over time step, m3)
    :return: withdrawals (to meet demand over time step at reservoir, m3)
    '''

    # Compute water availability, accounting for dead storage (volume units)
    water_available = storage_beg - reservoir.dead_storage + inflows

    # Release for downstream demand (volumetric rate)
    release = np.min([water_available, downstream_demand])

    # Update water availability
    water_available = water_available - release

    # Withdrawals from the reservoir
    withdrawals = np.min([water_available, site_demand])

   # Update water availability
    water_available = water_available - withdrawals

    # Check if reservoir is over full
    if water_available + reservoir.dead_storage > reservoir.full_lake_volume:
        # Lake is full
        storage_end = reservoir.full_lake_volume
        # Excess storage is spilled
        release = release + (water_available + reservoir.dead_storage - reservoir.full_lake_volume)
    else:
        # Lake is not full so water availability determines new storage
        storage_end = water_available + reservoir.dead_storage

    return storage_end, release, withdrawals

**We can test this function in three cases, (1) not enough water, (2) enough water, (3) too much water.**

Note the time step is not specified, so we can test with arbitray amounts of water and numbers!

So we call the function and check the answer is what we would expect! Below is a test for case 2 (the easiest), can you do the same for cases 1 and 3?

In [ ]:
# Inputs
storage_beg=0.8*res.full_lake_volume
inflows=0.2*res.full_lake_volume
site_demand=0.1*res.full_lake_volume
downstream_demand=0.2*res.full_lake_volume

# Call the function
x = single_step(res, storage_beg, inflows, site_demand, downstream_demand)

# Examine results
print('At the end of the time step, reservoir is ' + str(int(x[0] / res.full_lake_volume*100)) + '% full.')
print('At the end of the time step, releases meet ' + str(int(x[1] / downstream_demand * 100)) + '% of downstream demand.')
print('These releases correspond to '+ str(int(x[1] / res.full_lake_volume*100)) + '% of total storage volume.')
print('At the end of the time step, withdrawals directly from the reservoir meet ' + str(int(x[2] / site_demand * 100)) + '% of demand.')
print('These withdrawals correspond to '+ str(int(x[2] / res.full_lake_volume*100)) + '% of total storage volume.')

# Water balance over time

Now we start the water balance. First we need to extract the inflows.

## Inflow data

Let us first download the data. Note that we make the first column the index.

In [ ]:
inflow_data = pd.read_excel('data/Conowingo_data.xlsx', sheet_name='Flow data', index_col=0)
print(inflow_data.head(5))

Here's how we can create a time series of the total inflows, similar to tutorial 1. Note how the summation is made.

In [ ]:
print(inflow_data.sum(axis=1))

But the unit is in cubic feet per second (cfs), and it would be much better to work with metric units only... The conversion factor is 1ft = 0.3048m.

Let us now create a new Pandas DataFrame where each column is an element of the water balance. Our first column will be the total inflows.


In [ ]:
water_balance = pd.DataFrame(inflow_data.sum(axis=1) * 0.3048**3, columns = ['Total inflows (m3/s)'])
print(water_balance)

## Demand data

Now we can upload the demand data.

In [ ]:
demand_data = pd.read_excel('data/Conowingo_data.xlsx', sheet_name='Demands')
print(demand_data)

We need to first add these demands to the reservoir, by declaring them using the methods introduced with the Reservoir class.

Note the Chester, Baltimore and nuclear plant demands are extractions from the reservoir, whereas environmental flows are releases.

In [ ]:
res.add_downstream_demand('Environmental')
res.add_on_site_demand('Baltimore')
res.add_on_site_demand('Chester')
res.add_on_site_demand('Nuclear plant')

# Check result
print(res.demand_on_site)
print(res.demand_downstream)

Now we need to add these demands to the water balance. **Two key things to consider here, (1) demands need to be expressed for each day over 70 years, and (2) we must use SI units!**

In [ ]:
# First, initialise demand data into water balance: start with environmental demand
water_balance['Environmental demand (m3/s)'] = np.zeros(len(water_balance))
print(water_balance)

In [ ]:
# Now add the others
water_balance['Baltimore demand (m3/s)'] = np.zeros(len(water_balance))
water_balance['Chester demand (m3/s)'] = np.zeros(len(water_balance))
water_balance['Nuclear plant demand (m3/s)'] = np.zeros(len(water_balance))

In [ ]:
print(water_balance)

In [ ]:
# Then let's make a list of months to loop on them
months = np.arange(1,13,1)
print(months)

In [ ]:
# Loop on months
for month in months:

  # Make a mask to only keep the days that correspond to the current month.
  monthly_mask = water_balance.index.month == month

  # For all days of that month, get the correct data
  water_balance.loc[monthly_mask, 'Environmental demand (m3/s)'] = demand_data.iloc[month-1, 4] * 0.3048**3
  water_balance.loc[monthly_mask, 'Baltimore demand (m3/s)'] = demand_data.iloc[month-1, 1] * 0.3048**3
  water_balance.loc[monthly_mask, 'Chester demand (m3/s)'] = demand_data.iloc[month-1, 2] * 0.3048**3
  water_balance.loc[monthly_mask, 'Nuclear plant demand (m3/s)'] = demand_data.iloc[month-1, 3] * 0.3048**3

print(water_balance)

## Let's perform the water balance

This is a loop on days during the time frame of the simulation. At the heart is the one-step water balance.
**Mind the units: everything needs to be expressed in the same units (volume is easiest).**

**Question: what happens to the "available water" variable through this function? What is the role of minimum (or dead) storage? **



In [ ]:
def basic_water_balance(reservoir, water_flows):

  '''
  This function performs the water balance. Arguments are:
        reservoir: an instance of the Reservoir class
        water_flows: a pandas DataFrame that must contain inflows and demands.
  The function returns an updated water_flows DataFrame.
  '''


  # Local variable: number of time steps
  t_total = len(water_flows)

  # Local variable: number of seconds in a day
  n_sec = 86400

  # For computing efficiency: convert flows to numpy arrays outside of time loop

  # Inflows (in m3)
  inflows = water_flows['Total inflows (m3/s)'].to_numpy() * n_sec

  # Total downstream demand (in m3)
  downstream_demands = np.zeros(len(water_flows))
  for i in range(len(reservoir.demand_downstream)):
      # Get column with that demand
      demand_col = ([col for col in water_flows.columns if reservoir.demand_downstream[i] in col])
      # Add this demand to total demand
      downstream_demands = downstream_demands + water_flows.loc[:, demand_col[0]].to_numpy()
  downstream_demands = downstream_demands * n_sec  # conversion to m3

  # Total at-site demands (in m3)
  at_site_demands = np.zeros(len(water_flows))
  for i in range(len(reservoir.demand_on_site)):
     # Get column with that demand
     demand_col = ([col for col in water_flows.columns if reservoir.demand_on_site[i] in col])
     at_site_demands = at_site_demands + water_flows.loc[water_flows.index, demand_col[0]].to_numpy()
  at_site_demands = at_site_demands * n_sec  # conversion to m3

  # Initialise outputs
  # Storage needs to account for initial storage
  storage = np.zeros(t_total + 1)
  storage[0] = reservoir.initial_storage
  # Initialise at-site withdrawals and release as water balance components
  withdrawals = np.zeros(t_total)
  release = np.zeros(t_total)

  # Main loop
  for t in range(t_total):

    wb_out = single_step(reservoir, storage[t], inflows[t], at_site_demands[t], downstream_demands[t])
    storage[t+1] = wb_out[0]
    release[t] = wb_out[1]
    withdrawals[t] = wb_out[2]

  # Insert data into water balance (mind the flow rates conversions back into m3/s)
  water_flows['Withdrawals (m3/s)'] = withdrawals / n_sec
  water_flows['Release (m3/s)'] = release / n_sec
  water_flows['Storage (m3)'] = storage[1:]

  return water_flows


Now we call this water balance function for our case-study!

In [ ]:
basic_water_balance(res, water_balance)

We can plot results. We can also zoom in on any period of interest!
**Can you zoom in on that dry period in the 1960s?**

In [ ]:
# Storage over time
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
s, = ax.plot(water_balance.index, water_balance['Storage (m3)'], c='b', linewidth=2, label='Storage')
smin, = ax.plot(water_balance.index, res.dead_storage*np.ones(len(water_balance)), c='black', linestyle='--', linewidth=2, label='Dead storage')
legend = ax.legend(handles=[s, smin], loc=4)
ax.set_xlabel('Date', size=14)
ax.set_ylabel('Storage (m3)', size=14)

# We set the boundaries of the x-axis
# We can get the full period
ax.set_xlim(water_balance.index[0], water_balance.index[-1])

In [ ]:
# Or part of it!
ax.set_xlim(datetime.date(1962,1,1), datetime.date(1968,1,1))
fig

In [ ]:
# What happens to inflows and release during that period?
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ins, = ax.plot(water_balance.index, water_balance['Total inflows (m3/s)'], c='b', linewidth=2, label='Inflows')
outs, = ax.plot(water_balance.index, water_balance['Release (m3/s)'], c='r', linewidth=2, label='Release')
legend = ax.legend(handles=[ins, outs], loc=1)
ax.set_xlabel('Date', size=14)
ax.set_ylabel('Flow (m3/s)', size=14)

# We set the boundaries of the x-axis
ax.set_xlim(datetime.date(1962,1,1), datetime.date(1968,1,1))

# And adjust the y-axis (UNCOMMENT)
ax.set_ylim(0, 15000)

In [ ]:
# And what about the withdrawals from the reservoir?
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(water_balance.index, water_balance['Withdrawals (m3/s)'], c='b', linewidth=2)
ax.set_xlabel('Date', size=14)
ax.set_ylabel('Withdrawals (m3/s)', size=14)

# We set the boundaries of the x-axis
ax.set_xlim(datetime.date(1962,1,1), datetime.date(1968,1,1))
ax.set_ylim(0, 16)

**Questions:**


*   From the above figures and the water balance, when are withdrawals less than the demand?
*   Are the withdrawals large compared with the average inflows?
*   Can this reservoir store water from the wet season for the dry season? (hint: how long does it take to fill in the reservoir with 1000 m3/s inflows)



In [ ]:
# Let's save these withdrawals for future reference
basic_withdrawals = pd.Series(water_balance['Withdrawals (m3/s)'])

# Refinements of the water balance

Now we assume each withdrawal from the reservoir is at a given intake height (see the reservoir's key characteristics, as well as tutorial slide 3).

We will create a demand class to reflect the fact that demands have different characteristics.

In [ ]:
class Demand:

  # Define attributes specific to each Demand object
  def __init__(self, name, intake_depth=np.inf):
    self.name = name
    # intake depth from full lake level
    self.intake_depth = intake_depth

Now we need to re-initiatlise the demands from the reservoirs to add the intake heights to the demands.

In [ ]:
# Re-initialise
res.demand_downstream = []
res.demand_on_site = []

# Add demands using Demand objects
res.add_downstream_demand(Demand('Environmental'))
res.add_on_site_demand(Demand('Baltimore', key_parameters.iloc[6, 1]))
res.add_on_site_demand(Demand('Chester', key_parameters.iloc[7, 1]))
res.add_on_site_demand(Demand('Nuclear plant', key_parameters.iloc[6, 1]))

We also need to remove columns for withdrawals, release, storage, from the water balance DataFrame.

In [ ]:
water_balance.drop(['Release (m3/s)', 'Withdrawals (m3/s)', 'Storage (m3)'], axis=1, inplace=True)

# list the remaining columns
print(water_balance.columns)

**And that is going to change how we conduct the water balance! First, because we now need to differenciate between the different intakes.**

Let us update the one-step balance:

In [ ]:
def single_step_refined(reservoir, storage_beg, inflows, site_demand, downstream_demand):

    '''
    Note all in m3.
    :param reservoir: Object of the Reservoir class
    :param storage_beg: Initial storage at the beginning of the time step (m3)
    :param inflows: Inflows over the time step (m3)
    :param site_demand: Demand for withdrawal from reservoir over the time step (m3). Vector with length the number of demands
    :param downstream_demand: Demand for release for downstream use over the time step (m3)
    :return: storage_end (end of time step storage, m3)
    :return: release (amount of water released over time step, m3)
    :return: withdrawals (to meet demand over time step at reservoir, m3)
    '''

    # Compute water availability, accounting for dead storage (volume units)
    water_available = storage_beg - reservoir.dead_storage + inflows

    # Release for downstream demand (volumetric rate)
    release = np.min([water_available, downstream_demand])

    # Update water availability
    water_available = water_available - release

    # Height of water available in the reservoir, computed with height=0 when reservoir is empty
    height = reservoir.get_height(water_available + reservoir.dead_storage)

    # Initialise withdrawals FOR EACH DEMAND SOURCE
    withdrawals = np.zeros(len(reservoir.demand_on_site))

    # Compute on-site withdrawals FOR EACH DEMAND SOURCE
    for i in range(len(reservoir.demand_on_site)):

        # Check abstraction is possible
        if height + reservoir.demand_on_site[i].intake_depth > reservoir.total_lake_depth:
            # Withdrawals for downstream demand (volumetric rate)
            withdrawals[i] = np.min([water_available, site_demand[i]])
            # Update water availability
            water_available = water_available - withdrawals[i]

    # Check if reservoir is over full
    if water_available + reservoir.dead_storage > reservoir.full_lake_volume:
        # Lake is full
        storage_end = reservoir.full_lake_volume
        # Excess storage is spilled
        release = release + (water_available + reservoir.dead_storage - reservoir.full_lake_volume)
    else:
        # Lake is not full so water availability determines new storage
        storage_end = water_available + reservoir.dead_storage

    return storage_end, release, withdrawals

In [ ]:
def final_water_balance(reservoir, water_flows):

    """
    This function performs the water balance. Arguments are:
        reservoir: an instance of the Reservoir class
        water_flows: a pandas DataFrame that must contain inflows and demands.
    The function returns an updated water_flows DataFrame.
    """

    # Local variable: number of time steps
    t_total = len(water_flows)

    # Local variable: number of seconds in a day
    n_sec = 86400

    # For computing efficiency: convert flows to numpy arrays outside of time loop

    # Inflows (in m3)
    inflows = water_flows['Total inflows (m3/s)'].to_numpy() * n_sec

    # Total downstream demand (in m3)
    downstream_demands = np.zeros(len(water_flows))
    for i in range(len(reservoir.demand_downstream)):
        # Get column with that demand
        demand_col = ([col for col in water_flows.columns if reservoir.demand_downstream[i].name in col])
        # Add this demand to total demand
        downstream_demands = downstream_demands + water_flows.loc[:, demand_col[0]].to_numpy()
    downstream_demands = downstream_demands * n_sec  # conversion to m3

    # Total at-site demands (in m3)
    at_site_demands = np.zeros((len(water_flows), len(reservoir.demand_on_site)))
    for i in range(len(reservoir.demand_on_site)):
        # Get column with that demand
        demand_col = ([col for col in water_flows.columns if reservoir.demand_on_site[i].name in col])
        at_site_demands[:, i] = water_flows.loc[water_flows.index, demand_col[0]]
    at_site_demands = at_site_demands * n_sec  # conversion to m3

    # Initialise outputs
    # Storage needs to account for initial storage
    storage = np.zeros(t_total + 1)
    storage[0] = reservoir.initial_storage
    # Initialise at-site withdrawals and release as water balance components
    withdrawals = np.zeros((t_total, len(reservoir.demand_on_site)))
    release = np.zeros(t_total)

    # Main loop
    for t in range(t_total):

        wb_out = single_step_refined(reservoir, storage[t], inflows[t], at_site_demands[t, :], downstream_demands[t])
        storage[t+1] = wb_out[0]
        release[t] = wb_out[1]
        withdrawals[t, :] = wb_out[2]

    # Insert data into water balance (mind the flow rates conversions back into m3/s)
    for i in range(withdrawals.shape[1]):
        water_flows['Withdrawals ' + reservoir.demand_on_site[i].name + ' (m3/s)'] = withdrawals[:, i] / n_sec
    water_flows['Release (m3/s)'] = release / n_sec
    water_flows['Storage (m3)'] = storage[1:]

    return water_flows

In [ ]:
# Run this
final_water_balance(res, water_balance)

In [ ]:
# And what about the withdrawals from the reservoir?
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ol, = ax.plot(water_balance.index, basic_withdrawals, c='b', label='No intake')
ne, = ax.plot(water_balance.index, water_balance['Withdrawals Baltimore (m3/s)'] + water_balance['Withdrawals Chester (m3/s)'] + water_balance['Withdrawals Nuclear plant (m3/s)'], c='black', label='With intakes')
ax.set_xlabel('Date', size=14)
ax.set_ylabel('Withdrawals (m3/s)', size=14)
legend = ax.legend(handles=[ol, ne], loc=4)

# We set the boundaries of the x-axis
ax.set_xlim(datetime.date(1962,1,1), datetime.date(1968,1,1))
ax.set_ylim(0, 16)

**Question: did withdrawals increase or decrease compared with the previous water balance, and why?**

